<a href="https://colab.research.google.com/github/Suresh045/TNSDC/blob/main/SVR_Grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing the Libraies
import numpy as np
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('insurance_pre.csv')

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
indep=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
dep=dataset['charges']

In [7]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Kernel type
    'C': [1, 10, 100, 1000],                         # Regularization
    'gamma': ['scale', 'auto'],                      # Kernel coefficient
    'degree': [2, 3, 4],                             # Only used with 'poly'
    'epsilon': [0.1, 0.2, 0.5]                       # Error margin (for SVR)
}

grid = GridSearchCV(SVR(), param_grid, refit = True, verbose = 3 ,n_jobs=-1)
# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


GridSearchCV(estimator=SVR(), n_jobs=-1,
             param_grid={'C': [1, 10, 100, 1000], 'degree': [2, 3, 4],
                         'epsilon': [0.1, 0.2, 0.5], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             verbose=3)

In [11]:
from sklearn.model_selection import GridSearchCV

# Assuming 'grid' is your GridSearchCV object and has been fitted to the data
print("Best parameters:", grid.best_params_)

# Access the mean test score for the best parameters
best_index = grid.best_index_
best_r2_score = grid.cv_results_['mean_test_score'][best_index]
# Printing the best R² score
print("r2:",best_r2_score)

Best parameters: {'C': 1000, 'degree': 3, 'epsilon': 0.5, 'gamma': 'scale', 'kernel': 'poly'}
r2: 0.7878118909454586


In [12]:
table=pd.DataFrame.from_dict(grid.cv_results_)

In [13]:
table[['rank_test_score', 'mean_test_score', 'params']].sort_values('rank_test_score')

,rank_test_score,mean_test_score,params
257,1,0.787812,"{'C': 1000, 'degree': 3, 'epsilon': 0.5, 'gamm..."
249,2,0.787809,"{'C': 1000, 'degree': 3, 'epsilon': 0.2, 'gamm..."
241,3,0.787808,"{'C': 1000, 'degree': 3, 'epsilon': 0.1, 'gamm..."
261,4,0.787510,"{'C': 1000, 'degree': 3, 'epsilon': 0.5, 'gamm..."
253,5,0.787501,"{'C': 1000, 'degree': 3, 'epsilon': 0.2, 'gamm..."
...,...,...,...
10,283,-0.096206,"{'C': 1, 'degree': 2, 'epsilon': 0.2, 'gamma':..."
34,283,-0.096206,"{'C': 1, 'degree': 3, 'epsilon': 0.2, 'gamma':..."
2,286,-0.096207,"{'C': 1, 'degree': 2, 'epsilon': 0.1, 'gamma':..."
26,286,-0.096207,"{'C': 1, 'degree': 3, 'epsilon': 0.1, 'gamma':..."


In [14]:
# Assuming 'grid' is your GridSearchCV object that has been fitted to the data
best_model = grid.best_estimator_

In [15]:
import pickle
filename="SVR_G.sav"
pickle.dump(best_model,open(filename,'wb'))